In [1]:
import sys, os
sys.path.append(os.path.abspath("..")) 

## 1. 重复码（Repetition Code）

**重复码是最简单的纠错码，用来纠正比特翻转错误（bit-flip error）。**

核心思想：
- 将一个比特 m 重复 n 次，例如 n=3：
  - 0 → 000
  - 1 → 111
- 信道中若发生 1 或 2 个比特错误，只要多数表决即可恢复原本比特。

解码过程（n=3）：
- 000 → 0
- 001 → 0
- 010 → 0
- 100 → 0
- 011 → 1
- 101 → 1
- 110 → 1
- 111 → 1

重复码的纠错能力：
- 最多可以纠正 `(n-1)/2` 个 bit-flip 错误。

下面代码展示如何创建重复码、编码、注入错误并解码。

In [2]:
#重复码
from classical.repetition import create_repetition_code
import numpy as np

code = create_repetition_code(n=3)

m = 1
c = code.encode(m)
print("原始消息:", m)
print("编码后码字:", c)

# 加一个随机噪声
y = code.add_random_noise(c, p=0.2)
print("加入噪声后的码字: ", y)

m_hat, c_hat = code.decode(y)

print("解码得到的估计码字: ", c_hat)
print("解码得到的估计消息", m_hat)

print("\n消息是否恢复正确？", np.array_equal(m, m_hat))
print("码字是否恢复正确？", np.array_equal(c, c_hat))

原始消息: 1
编码后码字: [1 1 1]
加入噪声后的码字:  [1 1 1]
解码得到的估计码字:  [1 1 1]
解码得到的估计消息 1

消息是否恢复正确？ True
码字是否恢复正确？ True


## 2. 汉明码 Hamming(7,4) 

Hamming(7,4) 是经典纠错码中最著名的线性分组码之一：
- 输入：4 个数据位
- 输出：7 个编码位（添加 3 个校验位）
- 能够纠正 **任意 1 bit 错误**，能检测 **2 bit 错误**。

编码结构：
- 生成矩阵 G 用来将信息位映射到编码空间
- 奇偶校验矩阵 H 用于 syndrome 计算

解码关键：
- syndrome = H·r^T（收到向量乘以 H）
- syndrome 的结果唯一定位错误位

Hamming 码是最小的单比特纠错码，非常适合教学。
下面代码将演示 Hamming 码如何编码、注入 1 bit 错误并自动纠错。

In [3]:
#汉明码(7,4)
from classical.hamming74 import create_hamming_74, add_single_bit_error
import numpy as np

code = create_hamming_74()

m = np.array([1,0,1,1])
c = code.encode(m)

y = add_single_bit_error(c, position=4)

m_hat, c_hat, s, e_hat = code.decode(y)
print("原始消息:", m)
print("编码后码字:", c)
print("加入单比特错误后的码字:", y)
print("综合征 s = y H^T (mod 2):", s)
print("解码器认为的错误向量:", e_hat)
print("解码得到的估计码字:", c_hat)
print("解码得到的估计消息:", m_hat)

print("\n消息是否恢复正确？", np.array_equal(m, m_hat))
print("码字是否恢复正确？", np.array_equal(c, c_hat))

原始消息: [1 0 1 1]
编码后码字: [1 0 1 1 0 1 0]
加入单比特错误后的码字: [1 0 1 1 1 1 0]
综合征 s = y H^T (mod 2): [1 0 0]
解码器认为的错误向量: [0 0 0 0 1 0 0]
解码得到的估计码字: [1 0 1 1 0 1 0]
解码得到的估计消息: [1 0 1 1]

消息是否恢复正确？ True
码字是否恢复正确？ True


## 3. BCH (15,11) 

BCH 码是一类广泛使用的强纠错能力的循环码。

BCH(15,11) 特点：
- 输入：11 个数据位
- 输出：15 个编码位
- 能纠正：**最多 1 bit 错误**（t=1）

特点：
- 属于循环码（cyclic code）
- 使用生成多项式 g(x) 构造编码器
- Syndrome 多项式可以帮助找到错误位置

下面代码演示 BCH(15,11) 的编码与解码流程。


In [4]:
from classical.bch_15_11 import create_bch_15_11
import numpy as np

code = create_bch_15_11()

m = np.array([1,0,1,1,0,1,0,1,1,0,1])
c = code.encode(m)
print("原始消息:", m)
print("编码后的码字:", c)

# 随机加 1 比特错误
y = c.copy()
y[3] ^= 1
print("加入单比特错误后的码字:", y)
m_hat, c_hat, s, e_hat = code.decode(y)

print("综合征:", s)
print("解码器估计的错误向量:", e_hat)
print("解码得到的估计码字:", c_hat)
print("解码得到的估计消息:", m_hat)

print("\n消息是否恢复正确？", np.array_equal(m, m_hat))
print("码字是否恢复正确？", np.array_equal(c, c_hat))


原始消息: [1 0 1 1 0 1 0 1 1 0 1]
编码后的码字: [1 0 1 1 0 1 0 1 1 0 1 0 1 1 0]
加入单比特错误后的码字: [1 0 1 0 0 1 0 1 1 0 1 0 1 1 0]
综合征: [0 1 1 1]
解码器估计的错误向量: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
解码得到的估计码字: [1 0 1 1 0 1 0 1 1 0 1 0 1 1 0]
解码得到的估计消息: [1 0 1 1 0 1 0 1 1 0 1]

消息是否恢复正确？ True
码字是否恢复正确？ True


## 4. Reed-Muller（RM）

Reed-Muller 码是一类结构非常优雅、易于构造的线性码，广泛用于：
- 深空通信
- 嵌入式系统
- 量子纠错（如 CSS 码结构）

RM(1,3) 特点：
- 输入：8 个数据位
- 输出：16 位编码
- 纠错能力较强，且有明显的层次结构

下面代码演示 RM(1,3) 的调用方法。


In [5]:
from classical.reed_muller import create_rm_1_3
import numpy as np

rm_1_3 = create_rm_1_3()

# 随机生成一条消息
rng = np.random.default_rng(0)
m = rng.integers(0, 2, size=code.k)
print("原始消息 m:", m)

# 编码
c = code.encode(m)
print("编码后码字 c:", c)

# 在某一位上引入 1 比特错误
error_pos = 3
y = c.copy()
y[error_pos] ^= 1
print(f"在位置 {error_pos} 引入错误后的接收向量 y:", y)

# 解码
m_hat, c_hat, s, e_hat = code.decode(y)

print("\n综合征 s:", s)
print("估计错误向量 e_hat:", e_hat)
print("解码得到的消息 m_hat:", m_hat)
print("解码得到的码字 c_hat:", c_hat)

print("\n消息是否恢复正确？", np.array_equal(m, m_hat))
print("码字是否恢复正确？", np.array_equal(c, c_hat))


原始消息 m: [1 1 1 0 0 0 0 0 0 1 1]
编码后码字 c: [1 1 1 0 0 0 0 0 0 1 1 1 0 0 0]
在位置 3 引入错误后的接收向量 y: [1 1 1 1 0 0 0 0 0 1 1 1 0 0 0]

综合征 s: [0 1 1 1]
估计错误向量 e_hat: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
解码得到的消息 m_hat: [1 1 1 0 0 0 0 0 0 1 1]
解码得到的码字 c_hat: [1 1 1 0 0 0 0 0 0 1 1 1 0 0 0]

消息是否恢复正确？ True
码字是否恢复正确？ True
